In [17]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split

import pandas as pd
import os

In [18]:
ROOT_DIR = "../data"
RANDOM_STATE = 200

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

normal_ratio = 1.0

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)

## under sampling
df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)

train_data = df_concat

# df_train, df_val = train_test_split(
#     df_concat,
#     test_size=0.3,
#     stratify=df_concat["target"],
#     random_state=RANDOM_STATE,
# )

In [19]:
columns = ['1st Pressure Collect Result_AutoClave',
 'Production Qty Collect Result_Dam',
 'Production Qty Collect Result_Fill2',
 'Production Qty Collect Result_Fill1',
 '2nd Pressure Collect Result_AutoClave',
 'PalletID Collect Result_Fill1',
 'PalletID Collect Result_Fill2',
 'Chamber Temp. Collect Result_AutoClave',
 '3rd Pressure Collect Result_AutoClave',
 'PalletID Collect Result_Dam',
 'Receip No Collect Result_Fill2',
 'Receip No Collect Result_Fill1',
 'Receip No Collect Result_Dam',
 'Machine Tact time Collect Result_Fill2',
 'Machine Tact time Collect Result_Fill1',
 'Machine Tact time Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
 'Dispense Volume(Stage2) Collect Result_Fill1',
 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
 'Dispense Volume(Stage3) Collect Result_Fill1',
 'Dispense Volume(Stage1) Collect Result_Fill1',
 'Dispense Volume(Stage2) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
 'Dispense Volume(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
 'Chamber Temp. Unit Time_AutoClave',
 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
 'target']

train_data = train_data[columns]

In [20]:
setup_clf = setup(data=train_data, target="target", train_size=0.7, session_id=333, fold_shuffle=True)

,Description,Value
0,Session id,333
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(4700, 41)"
5,Transformed data shape,"(4700, 41)"
6,Transformed train set shape,"(3290, 41)"
7,Transformed test set shape,"(1410, 41)"
8,Numeric features,40
9,Preprocess,True


In [21]:
model = compare_models(sort='F1', fold=10, n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6070,0.6285,0.6070,0.6077,0.6063,0.2139,0.2147,0.1710
et,Extra Trees Classifier,0.5997,0.6317,0.5997,0.5998,0.5996,0.1994,0.1995,0.0570
ada,Ada Boost Classifier,0.5985,0.6255,0.5985,0.5990,0.5980,0.1969,0.1974,0.0440
rf,Random Forest Classifier,0.5970,0.6344,0.5970,0.5971,0.5968,0.1939,0.1940,0.0830
lightgbm,Light Gradient Boosting Machine,0.5927,0.6293,0.5927,0.5931,0.5923,0.1854,0.1857,0.1970
lda,Linear Discriminant Analysis,0.5824,0.6074,0.5824,0.5828,0.5819,0.1647,0.1652,0.0130
lr,Logistic Regression,0.5781,0.6004,0.5781,0.5789,0.5770,0.1562,0.1570,0.2020
ridge,Ridge Classifier,0.5708,0.6033,0.5708,0.5714,0.5700,0.1416,0.1422,0.0070
nb,Naive Bayes,0.5650,0.5847,0.5650,0.5664,0.5625,0.1301,0.1314,0.0070
knn,K Neighbors Classifier,0.5495,0.5853,0.5495,0.5497,0.5493,0.0991,0.0992,0.0100


In [22]:
tuned_model = [tune_model(i) for i in model]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5745,0.6069,0.5745,0.5748,0.5739,0.1487,0.1491
1,0.6565,0.6798,0.6565,0.6601,0.6545,0.3127,0.3164
2,0.6079,0.6297,0.6079,0.6085,0.6072,0.2156,0.2163
3,0.6049,0.6547,0.6049,0.6066,0.6030,0.2094,0.2113
4,0.6231,0.6607,0.6231,0.6242,0.6222,0.2460,0.2471
5,0.6109,0.6392,0.6109,0.6114,0.6106,0.2220,0.2224
6,0.5866,0.6139,0.5866,0.5867,0.5866,0.1733,0.1733
7,0.6292,0.6624,0.6292,0.6297,0.6289,0.2585,0.2590
8,0.5957,0.6028,0.5957,0.5964,0.5952,0.1917,0.1923


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5775,0.5946,0.5775,0.5780,0.5766,0.1548,0.1554
1,0.6413,0.6666,0.6413,0.6448,0.6390,0.2823,0.2859
2,0.6079,0.6307,0.6079,0.6085,0.6072,0.2156,0.2163
3,0.6170,0.6520,0.6170,0.6220,0.6128,0.2336,0.2387
4,0.6049,0.6736,0.6049,0.6078,0.6019,0.2093,0.2124
5,0.6261,0.6519,0.6261,0.6293,0.6241,0.2526,0.2556
6,0.5805,0.6146,0.5805,0.5819,0.5791,0.1614,0.1626
7,0.6474,0.6678,0.6474,0.6493,0.6464,0.2951,0.2968
8,0.6018,0.6053,0.6018,0.6048,0.5992,0.2040,0.2068


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5258,0.5676,0.5258,0.5259,0.5258,0.0517,0.0517
1,0.6079,0.6429,0.6079,0.6107,0.6051,0.2154,0.2184
2,0.5684,0.6116,0.5684,0.5705,0.5646,0.1363,0.1386
3,0.5897,0.6251,0.5897,0.5923,0.5863,0.1789,0.1817
4,0.6049,0.6487,0.6049,0.6245,0.5880,0.2088,0.2280
5,0.5653,0.5878,0.5653,0.5653,0.5653,0.1306,0.1307
6,0.5502,0.5758,0.5502,0.5501,0.5501,0.1003,0.1003
7,0.6079,0.6320,0.6079,0.6127,0.6040,0.2163,0.2208
8,0.5805,0.5780,0.5805,0.5832,0.5776,0.1615,0.1639


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5745,0.5955,0.5745,0.5762,0.5717,0.1485,0.1504
1,0.6049,0.6627,0.6049,0.6063,0.6033,0.2094,0.2110
2,0.6140,0.6453,0.6140,0.6161,0.6120,0.2276,0.2299
3,0.5897,0.6548,0.5897,0.5910,0.5879,0.1790,0.1805
4,0.6353,0.6813,0.6353,0.6418,0.6307,0.2700,0.2767
5,0.6140,0.6476,0.6140,0.6160,0.6125,0.2283,0.2301
6,0.5745,0.6151,0.5745,0.5753,0.5735,0.1492,0.1499
7,0.6565,0.6823,0.6565,0.6576,0.6560,0.3132,0.3142
8,0.5897,0.5993,0.5897,0.5910,0.5884,0.1796,0.1808


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5653,0.5863,0.5653,0.5654,0.5653,0.1307,0.1308
1,0.6140,0.6491,0.6140,0.6146,0.6133,0.2278,0.2285
2,0.5805,0.6069,0.5805,0.5806,0.5805,0.1611,0.1611
3,0.6109,0.6520,0.6109,0.6117,0.6102,0.2217,0.2225
4,0.6261,0.6513,0.6261,0.6265,0.6258,0.2521,0.2525
5,0.6140,0.6287,0.6140,0.6150,0.6132,0.2282,0.2291
6,0.5562,0.6021,0.5562,0.5562,0.5562,0.1125,0.1125
7,0.6201,0.6619,0.6201,0.6201,0.6201,0.2401,0.2401
8,0.6140,0.5900,0.6140,0.6141,0.6139,0.2280,0.2281


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [23]:
tuned_model

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.0005, loss='log_loss', max_depth=9,
                            max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.4, min_samples_leaf=3,
                            min_samples_split=7, min_weight_fraction_leaf=0.0,
                            n_estimators=150, n_iter_no_change=None,
                            random_state=333, subsample=0.25, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0,
                      class_weight='balanced_subsample', criterion='entropy',
                      max_depth=9, max_features=1.0, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0,
                      min_samples_leaf=6, min_samples_split=7,
                      min_wei

In [26]:
blended_soft = blend_models(estimator_list=tuned_model[:2],
                       fold=10,
                       method="soft",
                       optimize="F1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5805,0.5962,0.5805,0.5812,0.5795,0.1608,0.1616
1,0.6353,0.6690,0.6353,0.6380,0.6332,0.2702,0.2731
2,0.6109,0.6311,0.6109,0.6114,0.6104,0.2217,0.2223
3,0.6231,0.6528,0.6231,0.6277,0.6194,0.2457,0.2505
4,0.6109,0.6734,0.6109,0.6145,0.6076,0.2214,0.2252
5,0.6261,0.6520,0.6261,0.6293,0.6241,0.2526,0.2556
6,0.5714,0.6147,0.5714,0.5728,0.5698,0.1432,0.1444
7,0.6444,0.6687,0.6444,0.6460,0.6435,0.2890,0.2905
8,0.6049,0.6052,0.6049,0.6077,0.6025,0.2101,0.2128


In [27]:
final_model = finalize_model(blended_soft)
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [28]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

train_columns = train_data.columns
test_data = test_data[train_columns]

df_test_x = test_data.drop(columns=['target'])

In [29]:
test_pred = predict_model(final_model, data=df_test_x)
test_pred

,1st Pressure Collect Result_AutoClave,Production Qty Collect Result_Dam,Production Qty Collect Result_Fill2,Production Qty Collect Result_Fill1,2nd Pressure Collect Result_AutoClave,PalletID Collect Result_Fill1,PalletID Collect Result_Fill2,Chamber Temp. Collect Result_AutoClave,3rd Pressure Collect Result_AutoClave,PalletID Collect Result_Dam,...,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1,Chamber Temp. Unit Time_AutoClave,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,prediction_label,prediction_score
0,0.315,195,195,195,0.316,13.000000,13.000000,54,0.498,13.000000,...,244.300003,483,282.149994,244.300003,1270.699951,458.799988,157.000000,282.149994,Normal,0.5042
1,0.304,14,14,14,0.583,57.400002,19.799999,56,0.600,60.500000,...,244.505005,363,274.200012,1323.099976,1271.800049,837.900024,458.299988,274.200012,Normal,0.5798
2,0.308,98,98,98,0.308,1.000000,1.000000,54,0.500,1.000000,...,244.199997,483,284.600006,244.199997,377.000000,458.500000,157.000000,284.600006,AbNormal,0.5557
3,0.301,14,14,14,0.302,68.300003,20.000000,52,0.500,73.199997,...,244.300003,452,282.149994,1323.099976,1271.800049,837.700012,458.799988,282.149994,Normal,0.5292
4,0.302,1,1,1,0.492,54.299999,19.799999,54,0.497,54.299999,...,244.399994,363,281.221985,431.100006,377.600006,838.400024,458.399994,281.221985,Normal,0.5326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0.307,14,14,14,0.308,72.900002,19.500000,50,0.497,73.000000,...,244.300003,483,282.149994,1323.099976,1271.800049,837.700012,458.799988,282.149994,Normal,0.5432
17357,0.302,12,12,12,0.492,57.099998,19.799999,53,0.497,57.700001,...,244.505005,363,280.894012,1323.099976,1271.800049,837.400024,457.799988,280.894012,Normal,0.5381
17358,0.308,4,4,4,0.308,53.200001,20.500000,54,0.497,53.799999,...,244.399994,363,282.360992,431.100006,377.600006,838.400024,458.299988,282.360992,AbNormal,0.5013
17359,0.303,117,117,117,0.304,1.000000,1.000000,54,0.498,1.000000,...,244.300003,510,284.799988,244.300003,377.000000,460.000000,157.500000,284.799988,AbNormal,0.6023


In [30]:
df_sub = pd.read_csv("../data/submission.csv")
df_sub["target"] = test_pred["prediction_label"]
df_sub.to_csv("../data/automl/submission-7.csv", index=False)